In [35]:
from sympy import *
from sympy.physics.units import *
from sympy.physics.units import convert_to

import pint
ureg = pint.UnitRegistry()

In [36]:
class ParameterCollection(dict):
    def __init__(self):
        self.declaredSymbols = set()
        
    def add(self, name, symbol, expression, unit, description = "none"):
        declaredSymbols = set(map(lambda symbol: str(symbol), list(self.declaredSymbols)))
        
        if symbol in declaredSymbols:
            raise ValueError('Error adding new parameter: Parameter symbol already exists! Choose another name.')
        
        newParameter = Parameter(name, symbol, expression, unit, description, self)
        self[name] = newParameter
        self[symbol] = newParameter
        self.declaredSymbols.add(newParameter.symbol)
        return newParameter
    
class Parameter:
    defaultPrecision = 3
    
    def __init__(self, name, symbol, expression, unit, description, parentCollection):
        self.parentCollection = parentCollection
        self.name = name
        self.symbol = Symbol(symbol)
        globals()[symbol] = self.symbol
        globals()[name] = self.symbol
        
        self.subscriptItalics = []
        self.description = description
        self.descriptionDisplayed = False
        self.unit = ureg(str(unit))
        self.expressionList = []

        self.appendExp(expression)
    
    def appendExp(self, expression):
        self.expressionList.append(sympify(expression, evaluate = False))
        self.dependencies = self.depsFromExp(self.exp())
        self.evaluate()

        if not self.checkDim():
            raise ValueError("Error adding new parameter: User-input units don't match with evaluated units! Please check the dimensionality.")

    def exp(self):
        return self.expressionList[-1]

    def depsFromExp(self, expression):
        unfilteredSymbols = set(expression.atoms(Symbol))
        declaredSymbols = self.parentCollection.declaredSymbols
        return unfilteredSymbols & declaredSymbols
    
    def unwrapExp(self):
        expression = self.exp()
        deps = self.depsFromExp(expression)
        while len(deps) != 0:
                expression = self.substitute(expression, deps)
                deps = self.depsFromExp(expression)
        return expression
    
    def substitute(self, expression, dependencies, forDisplay = False):
        subObject = {}
        for dep in dependencies:
            depName = str(dep)
            depParameter = self.parentCollection[depName]
            depObject = depParameter.exp()
            if forDisplay:
                depObject = UnevaluatedExpr(depParameter.quantity.magnitude)
            subObject[depName] = depObject
        return expression.subs(subObject)
        
    def evaluate(self, precision = defaultPrecision):
        expression = self.unwrapExp().n(precision)
        self.quantity = ureg(str(expression))

    def checkDim(self):
        return (self.quantity.dimensionality == self.unit.dimensionality)

    def redefine(self, newExpression):
        self.appendExp(newExpression)
        
P = ParameterCollection()
P.add("xCoord", "x_l", 20*mm, m)
P.add("yCoord", "y_l", x_l / 2, mm)
P.add("overallLength", "l", sqrt(xCoord ** 2 + yCoord ** 2), mm)

print(P["x_l"].dependencies)
print(P["y_l"].dependencies)
print(P["l"].dependencies)

print(P["x_l"].quantity)
print(P["y_l"].quantity)
print(P["l"].quantity)

display(P["l"].substitute(P["l"].exp(), P["l"].dependencies, true))

ValueError: Error adding new parameter: User-input units don't match with evaluated units! Please check the dimensionality.